In [ ]:
# Project

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
tranDtype = {'id': 'uint64',
 'chain': 'uint16',
 'dept': 'uint8',
 'category': 'uint16',
 'company': 'uint64',
 'brand': 'uint32',
#  'date' : 'object',
 'productsize': 'float32',
 'productmeasure': 'category',
 'purchasequantity': 'int64',
 'purchaseamount': 'float32'}

In [ ]:
import dask.dataframe as pd 
ef = dd.read_csv('/content/drive/MyDrive/1經濟學/專題/transactions.csv', dtype = tranDtype)
ef.head()

,id,chain,dept,category,company,brand,date,productsize,productmeasure,purchasequantity,purchaseamount
0,86246,205,7,707,1078778070,12564,2012-03-02,12.0,OZ,1,7.59
1,86246,205,63,6319,107654575,17876,2012-03-02,64.0,OZ,1,1.59
2,86246,205,97,9753,1022027929,0,2012-03-02,1.0,CT,1,5.99
3,86246,205,25,2509,107996777,31373,2012-03-02,16.0,OZ,1,1.99
4,86246,205,55,5555,107684070,32094,2012-03-02,16.0,OZ,2,10.38


In [ ]:
ef.info(memory_usage='deep')

In [ ]:
import pandas as pd
offers = pd.read_csv(r'/content/drive/MyDrive/1經濟學/專題/offers.csv')
# transactions = pd.read_csv(r'/content/drive/MyDrive/1經濟學/專題/newdata.csv')
trainhistory = pd.read_csv(r'/content/drive/MyDrive/1經濟學/專題/trainHistory.csv')
testhistory = pd.read_csv(r'/content/drive/MyDrive/1經濟學/專題/testHistory.csv')

In [ ]:
transactions.shape

(Delayed('int-a256edf8-7c72-4963-83df-9e1dd8d7ea11'), 11)

In [ ]:
def generateFeature(offers, transactions, trainHistory, testHistory, tidx) :
    # import useful package
    from datetime import timedelta
    import pandas as pd
    import numpy as np
    import time
    import gc
    ts = time.time()
    tk = 1
    
    use = pd.concat([trainHistory.drop(columns = ['repeater', 'repeattrips']), testHistory], axis=0, ignore_index=True)
    offers.rename(columns={'category' : 'offercategory', 'company' : 'offercompany', 'brand' : 'offerbrand'}, inplace = True)
    usf = use.set_index('offer').join(offers.set_index('offer'))[['id', 'offerdate', 'offercategory', 'offercompany', 'offerbrand']]
    nu = transactions.set_index('id').join(usf.set_index('id'))

    del usf, trainHistory, testHistory, transactions
    gc.collect()
    
    te = time.time()
    print('Inner -', str(tidx) + ' of ' + str(tk) + '(put offer information) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    tk+=1
    
    # generate time index
    day = np.linspace(30, 180, 6, endpoint = True).astype(int).astype(str)
    mea = ['total', 'company', 'brand', 'category']
    date_format = '%Y-%m-%d'
    nu['daydiff'] = pd.to_datetime(nu['offerdate'], format = date_format) - pd.to_datetime(nu['date'], format = date_format)
    for i in day :
        nu['diff_' + i] = nu['daydiff'] <= timedelta(days = int(i))
    for i in mea[1:] :
        nu['in_' + i] = nu['offer' + i] == nu[i]
    
    te = time.time()
    print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate time index) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    tk+=1
    
    # put offervalue
    of1 = offers[['offer', 'offervalue']]
    use = pd.merge(use, of1, on='offer')
    
    te = time.time()
    print('Inner -', str(tidx) + ' of ' + str(tk) + '(put offervalue) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    tk+=1
    
    # generate total
    new1 = nu.groupby(['id']).agg({'purchaseamount': ['sum', 'mean', 'count'], 'daydiff': ['min']})
    new2 = nu.groupby(['id', 'productmeasure'])['purchasequantity'].sum().reset_index('productmeasure')
    new = pd.concat([new1, new2.loc[(new2['productmeasure'].values == 'CT'), 'purchasequantity'],
                    new2.loc[(new2['productmeasure'].values == 'OZ'), 'purchasequantity'],
                    new2.loc[(new2['productmeasure'].values == 'LT'), 'purchasequantity']], axis = 1)
    new.columns = ['buy_total_amount', 'buy_total_avgamount', 'buy_total_freq',
                'buy_total_daydiff', 'buy_total_CT', 'buy_total_OZ', 'buy_total_LT']
    use = use.set_index('id').join(new)
    
    te = time.time()
    print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    tk+=1

    
    for i in day :
        filter = nu['diff_' + i]
        new1 = nu[filter].groupby('id').agg({'purchaseamount' : ['sum', 'mean', 'count']})
        new2 = nu[filter].groupby(['id', 'productmeasure'])['purchasequantity'].sum().reset_index('productmeasure')
        new = pd.concat([new1, new2[(new2['productmeasure'].values == 'CT')]['purchasequantity'],
                        new2[(new2['productmeasure'].values == 'OZ')]['purchasequantity'],
                        new2[(new2['productmeasure'].values == 'LT')]['purchasequantity']], axis = 1)
        new.columns = ['buy_total_amount_' + i, 'buy_total_avgamount_' + i, 'buy_total_freq_' + i,
                        'buy_total_CT_' + i, 'buy_total_OZ_' + i, 'buy_total_LT_' + i]
        use = use.join(new)

    te = time.time()
    print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total day) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    tk+=1
    
    # generate company & brand & category
    mea = ['company', 'brand', 'category']
    for m in mea :
        filter = nu['in_' + m]
        new1 = nu[filter].groupby(['id']).agg({'purchaseamount': ['sum', 'mean', 'count'], 'daydiff': ['min']})
        new2 = nu[filter].groupby(['id', 'productmeasure'])['purchasequantity'].sum().reset_index('productmeasure')
        new = pd.concat([new1, new2.loc[(new2['productmeasure'].values == 'CT'), 'purchasequantity'],
                        new2.loc[(new2['productmeasure'].values == 'OZ'), 'purchasequantity'],
                        new2.loc[(new2['productmeasure'].values == 'LT'), 'purchasequantity']], axis = 1)
        new.columns = ['buy_'+m+'_amount', 'buy_'+m+'_avgamount', 'buy_'+m+'_freq',
                        'buy_'+m+'_daydiff', 'buy_'+m+'_CT', 'buy_'+m+'_OZ', 'buy_'+m+'_LT']
        use = use.join(new)

    te = time.time()
    print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    tk+=1
    
    
    for i in day :
        for m in mea :
            filter = nu['diff_'+i] & nu['in_' + m]
            new1 = nu[nu['diff_'+i]].groupby('id').agg({'purchaseamount' : ['sum', 'mean', 'count']})
            new2 = nu[nu['diff_'+i]].groupby(['id', 'productmeasure'])['purchasequantity'].sum().reset_index('productmeasure')
            new = pd.concat([new1, new2[(new2['productmeasure'].values == 'CT')]['purchasequantity'],
                            new2[(new2['productmeasure'].values == 'OZ')]['purchasequantity'],
                            new2[(new2['productmeasure'].values == 'LT')]['purchasequantity']], axis = 1)
            new.columns = ['buy_'+m+'_amount_' + i, 'buy_'+m+'_avgamount_' + i, 'buy_'+m+'_freq_' + i,
                        'buy_'+m+'_CT_' + i, 'buy_'+m+'_OZ_' + i, 'buy_'+m+'_LT_' + i]
            use = use.join(new)
    
    te = time.time()
    print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other day) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    tk+=1
    
    
    # generate not buy index
    for m in mea :
        f = (use['buy_'+m+'_freq']>0) 
        use['not_buy_'+m] = True!=f
    
    it = np.repeat(range(3), 2)
    for m1, m2 in zip(it[:3], it[3:]) :
        n = use['not_buy_'+mea[m1]]
        t = use['not_buy_'+mea[m2]]
        use['not_buy_'+mea[m1]+'_'+mea[m2]] = (n & t)

    del new1, new2, new
    gc.collect()

    te = time.time()
    print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate not buy) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    tk+=1

    # handle na problem
    dayVar = ['buy_company_daydiff', 'buy_brand_daydiff', 'buy_category_daydiff', 'buy_total_daydiff']
    use[dayVar] = use[dayVar].fillna(timedelta(0)).astype('timedelta64[D]').astype(int)
    use = use.fillna(0)
    
    te = time.time()
    print('Inner -', str(tidx) + ' of ' + str(tk) + '(handle na problem) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    tk+=1
    
    # transform bool type into int
    use[use.columns[use.dtypes == 'bool']] = use[use.columns[use.dtypes == 'bool']].astype(int)

    te = time.time()
    print('Inner -', str(tidx) + ' (generate feature) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    
    return use

In [ ]:
filee = '/content/drive/MyDrive/1經濟學/專題/transactions.csv'
file1 = '/content/drive/MyDrive/1經濟學/專題/newSplit/'

In [ ]:
se = ['id', 'chain', 'dept', 'category', 'company', 'brand', 'date',
       'productsize', 'productmeasure', 'purchasequantity', 'purchaseamount']

In [ ]:
%%time
from datetime import datetime, date
import gc
big_size = 50000000
k = 0
start = datetime.now()
with pd.read_csv(filee, chunksize = big_size, dtype = tranDtype, 
                 header = None, skiprows = 1,
                #  parse_dates = ['date'], infer_datetime_format = True,
                 engine = 'c') as reader:
  for df in reader:
    k += 1
    df.columns = se
    use = generateFeature(offers, df, trainhistory, testhistory, k)
    use.to_csv(file1+'output'+str(k)+'.csv', index=False)
    del use
    gc.collect()
    print('Export : ', k, len(df), datetime.now() - start)

In [ ]:
df.columns

Index(['id', 'chain', 'dept', 'category', 'company', 'brand', 'date',
       'productsize', 'productmeasure', 'purchasequantity', 'purchaseamount'],
      dtype='object')

In [ ]:
%%time
group = transactions.groupby('id')

In [ ]:
%%time
a = group.first().compute()

CPU times: user 24.5 s, sys: 3.12 s, total: 27.6 s
Wall time: 12.5 s


In [ ]:
a.reset_index()['id']

0              86246
1              86252
2           12262064
3           12277270
4           12332190
             ...    
310660    4837642552
310661    4843417324
310662    4847628950
310663    4847787712
310664    4853598737
Name: id, Length: 310665, dtype: int64

In [ ]:
%%time
for i in group :
  print(i)

In [ ]:
group.get_group()

In [ ]:
%%time
a = []
for i in range(200) :
  a.append(group[i][1].values.astype(str))

KeyError: ignored

In [ ]:
i[1]

In [ ]:
%%time
# group = df.iloc[:100000,:].groupby('id')

In [ ]:
group.first()

In [ ]:
group = df.iloc[:100000, :].groupby('id')

In [ ]:
for i in group :
  print(1.get)

NotImplementedError: ignored

In [ ]:
%%time
k=0
for i in group :
  k+=1
  with open('output'+str(i[0])+'.csv', 'w') as out_f:
      for row in i[1].values.astype(str) :
          out_f.write(",".join(row) + '\n')

TypeError: ignored

90

In [ ]:
import csv

In [ ]:
%%time
k=0
for i in group :
  k+=1
  with open('output'+str(i[0])+'.csv', 'w', 'w') as csvfile:
      writer = csv.writer(csvfile)
      for row in i[1]:
          writer.writerow(row.encode())

In [ ]:
group = df.iloc[:100000, :].groupby('id')

In [ ]:
for i in a.reset_index()['id'] :
  print(1)

In [ ]:
%%time
for o in group.first().compute().reset_index()['id'] :
  i = group.get_group(o)
  i[1].to_csv('output'+str(i[0])+'.csv')

KeyboardInterrupt: ignored

In [ ]:
transactions_file = '/content/drive/MyDrive/1經濟學/專題/transactions.csv'
reduced_file = 'reducedddd.csv'

In [ ]:
%%time
with open(reduced_file, "wb") as outfile:
  reduced = 0
  for e, line in enumerate( open(transactions_file) ):
    id = line.split(",")[0]
    if e == 0:
      outfile.write(line.encode())
    else:
      if(line.split(",")[0]!=id) :
        
      outfile.write(line.encode())
      reduced += 1
    if e % 5000000 == 0:
      if reduced > 20000000 :
        break
print(e, reduced)

25000000 25000000
CPU times: user 14 s, sys: 4.35 s, total: 18.4 s
Wall time: 20.2 s


'222481657'

In [ ]:
df

,id,chain,dept,category,company,brand,date,productsize,productmeasure,purchasequantity,purchaseamount
0,86246,205,7,707,1078778070,12564,2012-03-02,12.00,OZ,1,7.59
1,86246,205,63,6319,107654575,17876,2012-03-02,64.00,OZ,1,1.59
2,86246,205,97,9753,1022027929,0,2012-03-02,1.00,CT,1,5.99
3,86246,205,25,2509,107996777,31373,2012-03-02,16.00,OZ,1,1.99
4,86246,205,55,5555,107684070,32094,2012-03-02,16.00,OZ,2,10.38
...,...,...,...,...,...,...,...,...,...,...,...
4999995,120912706,88,51,5123,102113020,15704,2012-03-23,26.40,OZ,2,12.99
4999996,120912706,88,56,5610,102113020,15704,2012-03-23,8.00,OZ,1,-1.20
4999997,120912706,88,33,3308,102840020,3336,2012-03-23,9.75,OZ,3,6.83
4999998,120912706,88,26,2634,104200040,14813,2012-03-23,12.00,RL,1,4.71


In [ ]:
use = generateFeature(offers, transactions, trainhistory, testhistory, 1)

In [ ]:
k

0

In [ ]:
group[0][1].values.astype(str)

In [ ]:
out_l = pd.DataFrame(group[0][1].values.astype(str))
out_l.info(memory_usage='deep')
out_l.shape

In [ ]:
k = 1
for i ,j in df.iterrows():
  if(k==1) :
    print("This is i : ", i, "This is j : ", j)
  k+=1

In [ ]:
offers_file = '/content/drive/MyDrive/1經濟學/專題/offers.csv'
transactions_file = '/content/drive/MyDrive/1經濟學/專題/transactions.csv'
reduced_file = '/content/drive/MyDrive/1經濟學/專題/reduceddd.csv'

In [ ]:
from datetime import datetime, date
import string, os
from collections import defaultdict
start = datetime.now()
#get all categories and comps on offer in a dict
offers_cat = {}
offers_co = {}
for e, line in enumerate( open(offers_file) ):
  offers_cat[ line.split(",")[1] ] = 1
  offers_co[ line.split(",")[3] ] = 1
#open output file
with open(reduced_file, "wb") as outfile:
  #go through transactions file and reduce
  reduced = 0
  for e, line in enumerate( open(transactions_file) ):
    if e == 0:
      print(line) #print header
    else:
      #only write when if category in offers dict
      if line.split(",")[3] in offers_cat or line.split(",")[4] in offers_co:
        outfile.write(line.encode())
        reduced += 1
    #progress
    if e % 5000000 == 0:
      print(e, reduced, datetime.now() - start)
print(e, reduced, datetime.now() - start)

In [ ]:
def generateFeature(offers, transactions, trainHistory, testHistory, tidx) :
    # import useful package
    from datetime import timedelta
    from itertools import cycle
    import pandas as pd
    import numpy as np
    import time
    import gc
    ts = time.time()
    # tk = 1
    
    # get all data & delete those not in transactions
    train = trainHistory.drop(columns = ['repeater', 'repeattrips'])
    data = pd.concat([train, testHistory], axis=0, ignore_index=True)
    use = data[data['id'].isin(transactions['id'])] # (310665, )

    del trainHistory, train, testHistory
    gc.collect()
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(get all data) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    # put offer information into transactions
    of = offers[['offer', 'category', 'company', 'offervalue', 'brand']]
    usf = pd.merge(use, of, on='offer')
    usf.columns = ['id', 'chain', 'offer', 'market', 'offerdate', 'offercategory', 'offercompany',
          'offervalue', 'offerbrand']
    tu = usf[['id', 'offer', 'offerdate', 'offercategory', 'offercompany', 'offerbrand']]
    nu = pd.merge(tu, transactions, on='id')
    
    del tu, usf, of, transactions
    gc.collect()
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(put offer information) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    # generate time index
    date_format = '%Y-%m-%d'
    nu['offerdate'] = pd.to_datetime(nu['offerdate'], format = date_format)
    nu['date'] = pd.to_datetime(nu['date'], format = date_format)
    nu['daydiff'] = nu['offerdate'] - nu['date']
    nu['diff_180'] = nu['offerdate'] - timedelta(days = 180)
    nu['diff_150'] = nu['offerdate'] - timedelta(days = 150)
    nu['diff_120'] = nu['offerdate'] - timedelta(days = 120)
    nu['diff_90'] = nu['offerdate'] - timedelta(days = 90)
    nu['diff_60'] = nu['offerdate'] - timedelta(days = 60)
    nu['diff_30'] = nu['offerdate'] - timedelta(days = 30)
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate time index) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    # put offervalue
    of1 = offers[['offer', 'offervalue']]
    use = pd.merge(use, of1, on='offer')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(put offervalue) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    # generate total
    group = nu.groupby(['id'])
    
    test = group['chain'].count().reset_index()
    test.columns = ['id', 'buy_total_freq']
    use = pd.merge(use, test, on='id')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total freq) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    test = group['purchaseamount'].sum().reset_index()
    test.columns = ['id', 'buy_total_amount']
    use = pd.merge(use, test, on='id')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total amount) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    test = group['purchaseamount'].mean().reset_index()
    test.columns = ['id', 'buy_total_avgamount']
    use = pd.merge(use, test, on='id')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total avg) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    test = group['purchasequantity'].sum().reset_index()
    test.columns = ['id', 'buy_total_quantity']
    use = pd.merge(use, test, on='id')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total quan) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    test = group['daydiff'].min().reset_index()
    test.columns = ['id', 'buy_total_daydiff']
    use = pd.merge(use, test, on='id')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    
    day = np.linspace(30, 180, 6, endpoint=True).astype(int).astype(str)
    
    for i in day :
      daa = 'diff_' + i
      nu['ascom'] = nu['date'] >= nu[daa]
    
      name = 'buy_total_amount_' + i
      var = 'purchaseamount'
    
      group = nu.groupby(['id', 'ascom'])
      test = group[var].sum().reset_index()
      test.columns = ['id', 'ascom', name]
      use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate day amount) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      name = 'buy_total_quantity_' + i
      var = 'purchasequantity'
    
      group = nu.groupby(['id', 'ascom'])
      test = group[var].sum().reset_index()
      test.columns = ['id', 'ascom', name]
      use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate day quan) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      name = 'buy_total_freq_' + i
      var = 'purchaseamount'
    
      group = nu.groupby(['id', 'ascom'])
      test = group['chain'].count().reset_index()
      test.columns = ['id', 'ascom', name]
      use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate day) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
    # generate company & brand & category
    mea = ['company', 'brand', 'category']
    day = np.repeat(np.linspace(30, 180, 6, endpoint=True), 3).astype(int).astype(str)
    
    for i in mea :
      nu['ascom'] = nu[i] == nu['offer' + i]
      group = nu.groupby(['id', 'ascom'])
    
      test = group['chain'].count().reset_index()
      test.columns = ['id', 'ascom', 'buy_'+i+'_freq']
      use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_freq']], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other freq) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      test = group['purchaseamount'].sum().reset_index()
      test.columns = ['id', 'ascom', 'buy_'+i+'_amount']
      use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_amount']], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other amount) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      test = group['purchaseamount'].mean().reset_index()
      test.columns = ['id', 'ascom', 'buy_'+i+'_avgamount']
      use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_avgamount']], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other avg) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      test = group['purchasequantity'].sum().reset_index()
      test.columns = ['id', 'ascom', 'buy_'+i+'_quantity']
      use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_quantity']], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other quan) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      test = group['daydiff'].min().reset_index()
      test.columns = ['id', 'ascom', 'buy_'+i+'_daydiff']
      use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_daydiff']], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
    
    for i, j in zip(day, cycle(mea)) :
      daa = 'diff_' + i
      nu['ascom'] = ((nu['date'] >= nu[daa]) & (nu[j] == nu['offer' + j]))
    
      name = 'buy_'+j+'_amount_' + i
      var = 'purchaseamount'
    
      group = nu.groupby(['id', 'ascom'])
      test = group[var].sum().reset_index()
      test.columns = ['id', 'ascom', name]
      use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other day amount) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      name = 'buy_'+j+'_quantity_' + i
      var = 'purchasequantity'
    
      group = nu.groupby(['id', 'ascom'])
      test = group[var].sum().reset_index()
      test.columns = ['id', 'ascom', name]
      use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    
      # te = time.time()
      # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other day quan) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # tk+=1
    
      name = 'buy_'+j+'_freq_' + i
      var = 'purchaseamount'
    
      group = nu.groupby(['id', 'ascom'])
      test = group['chain'].count().reset_index()
      test.columns = ['id', 'ascom', name]
      use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    
      # te = time.time()
      # # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other day) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
      # # tk+=1
    
    # generate not buy index
    nu['ascom'] = (nu['company'] == nu['offercompany']) & (nu['brand'] == nu['offerbrand']) & (nu['category'] == nu['offercategory'])
    group = nu.groupby(['id', 'ascom'])

    name1 = 'buy_company_brand_category'
    name = name1 + '_freq'

    test = group['chain'].count().reset_index()
    test.columns = ['id', 'ascom', name]
    use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    new = pd.DataFrame((use[name] > 0) != True)
    new.columns = ['not_' + name1]
    use = pd.concat([use, new], axis = 1)



    nu['ascom'] = (nu['company'] == nu['offercompany']) & (nu['brand'] == nu['offerbrand'])
    group = nu.groupby(['id', 'ascom'])

    name1 = 'buy_company_brand'
    name = name1 + '_freq'

    test = group['chain'].count().reset_index()
    test.columns = ['id', 'ascom', name]
    use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    new = pd.DataFrame((use[name] > 0) != True)
    new.columns = ['not_' + name1]
    use = pd.concat([use, new], axis = 1)



    nu['ascom'] = (nu['company'] == nu['offercompany']) & (nu['category'] == nu['offercategory'])
    group = nu.groupby(['id', 'ascom'])

    name1 = 'buy_company_category'
    name = name1 + '_freq'

    test = group['chain'].count().reset_index()
    test.columns = ['id', 'ascom', name]
    use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    new = pd.DataFrame((use[name] > 0) != True)
    new.columns = ['not_' + name1]
    use = pd.concat([use, new], axis = 1)



    nu['ascom'] = (nu['brand'] == nu['offerbrand']) & (nu['category'] == nu['offercategory'])
    group = nu.groupby(['id', 'ascom'])

    name1 = 'buy_brand_category'
    name = name1 + '_freq'

    test = group['chain'].count().reset_index()
    test.columns = ['id', 'ascom', name]
    use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
    new = pd.DataFrame((use[name] > 0) != True)
    new.columns = ['not_' + name1]
    use = pd.concat([use, new], axis = 1)

    del new
    gc.collect()

    new1 = pd.DataFrame((use['buy_company_freq'] > 0) != True)
    new1.columns = ['not_buy_company']
    new2 = pd.DataFrame((use['buy_brand_freq'] > 0) != True)
    new2.columns = ['not_buy_brand']
    new3 = pd.DataFrame((use['buy_category_freq'] > 0) != True)
    new3.columns = ['not_buy_category']
    use = pd.concat([use, new1, new2, new3], axis = 1)

    del new1, new2, new3
    gc.collect()

    # handle na problem
    dayVar = ['buy_company_daydiff', 'buy_brand_daydiff', 'buy_category_daydiff', 'buy_total_daydiff']
    use1 = use[dayVar]
    use.drop(columns = dayVar, inplace = True)
    use = use.fillna(0)
    use1 = use1.fillna(timedelta(0))
    use = pd.concat([use, use1], axis = 1)

    del use1
    gc.collect()
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(handle na problem) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    # transform date type into int
    for i in use[dayVar] :
      use[i] = use[i].astype('str').apply(lambda x:x[:-5]).astype('int32')
    
    # te = time.time()
    # print('Inner -', str(tidx) + ' of ' + str(tk) + '(transform date type) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    # tk+=1
    
    # transform bool type into int
    for i in use.columns :
      if(use[i].dtypes == 'bool') :
        use[i] = use[i].apply(int)
    
    te = time.time()
    print('Inner -', str(tidx) + ' (generate feature) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
    
    return use

In [ ]:
import pandas as pd
offers = pd.read_csv(r'/content/drive/MyDrive/1經濟學/專題/offers.csv')
transactions = pd.read_csv(r'/content/drive/MyDrive/1經濟學/專題/newdata.csv')
trainhistory = pd.read_csv(r'/content/drive/MyDrive/1經濟學/專題/trainHistory.csv')
testhistory = pd.read_csv(r'/content/drive/MyDrive/1經濟學/專題/testHistory.csv')

In [ ]:
%%time
# import useful package
from datetime import timedelta
from itertools import cycle
import pandas as pd
import numpy as np
import time
import gc
ts = time.time()
# tk = 1

# get all data & delete those not in transactions
# trainhistory.drop(columns = ['repeater', 'repeattrips'], inplace = True)
use = pd.concat([trainhistory.drop(columns = ['repeater', 'repeattrips']), testhistory], axis=0, ignore_index=True)
# use = data[data['id'].isin(transactions['id'])] # (310665, )
# usf = use.set_index('offer').join( \
#               offers.rename(columns={'category' : 'offercategory', 'company' : 'offercompany', 'brand' : 'offerbrand'}).set_index('offer'))
offers.rename(columns={'category' : 'offercategory', 'company' : 'offercompany', 'brand' : 'offerbrand'}, inplace = True)
usf = use.set_index('offer').join(offers.set_index('offer'))[['id', 'offerdate', 'offercategory', 'offercompany', 'offerbrand']]
nu = transactions.set_index('id').join(usf.set_index('id'))
del usf
gc.collect()

CPU times: user 3.1 s, sys: 2.61 s, total: 5.71 s
Wall time: 5.71 s


52

In [ ]:
a = nu.copy()

In [ ]:
pd.to_datetime(nu['offerdate'], format = date_format)

id
86246        2013-04-24
86246        2013-04-24
86246        2013-04-24
86246        2013-04-24
86246        2013-04-24
                ...    
4847628950   2013-07-24
4847787712   2013-07-26
4847787712   2013-07-26
4853598737   2013-07-27
4853598737   2013-07-27
Name: offerdate, Length: 15349956, dtype: datetime64[ns]

In [ ]:
%%time
date_format = '%Y-%m-%d'
new = []
nu['offerdate'] = pd.to_datetime(nu['offerdate'], format = date_format)
nu['date'] = pd.to_datetime(nu['date'], format = date_format)

CPU times: user 2.31 s, sys: 457 ms, total: 2.76 s
Wall time: 2.75 s


In [ ]:
new = []
new.append(nu['offerdate'] - nu['date'])

In [ ]:
for i in np.linspace(30, 180, 6, endpoint=True).astype(int).astype(str) :
  new.append(nu['offerdate'] - timedelta(days = int(i)))

In [ ]:
%%time
new = nu['offerdate'] - nu['date']
for i in np.linspace(30, 180, 6, endpoint=True) :
  pd.concat([new, nu['offerdate'] - timedelta(days = i)], axis = 1)  

CPU times: user 1.61 s, sys: 889 ms, total: 2.5 s
Wall time: 2.53 s


In [ ]:
new

id
86246        418 days
86246        418 days
86246        414 days
86246        413 days
86246        406 days
               ...   
4847628950     1 days
4847787712     5 days
4847787712     5 days
4853598737     7 days
4853598737     7 days
Length: 15349956, dtype: timedelta64[ns]

In [ ]:
%%time
date_format = '%Y-%m-%d'
new = []
nu['offerdate'] = pd.to_datetime(nu['offerdate'], format = date_format)
nu['date'] = pd.to_datetime(nu['date'], format = date_format)


nu['daydiff'] = nu['offerdate'] - nu['date']
new.columns = ['offerdate']
nu['diff_180'] = nu['offerdate'] - timedelta(days = 180)
new.columns = ['offerdate']
nu['diff_150'] = nu['offerdate'] - timedelta(days = 150)
new.columns = ['offerdate']
nu['diff_120'] = nu['offerdate'] - timedelta(days = 120)
new.columns = ['offerdate']
nu['diff_90'] = nu['offerdate'] - timedelta(days = 90)
new.columns = ['offerdate']
nu['diff_60'] = nu['offerdate'] - timedelta(days = 60)
new.columns = ['offerdate']
nu['diff_30'] = nu['offerdate'] - timedelta(days = 30)

AttributeError: ignored

In [ ]:
day = np.linspace(30, 180, 6, endpoint = True).astype(int).astype(str)

In [ ]:
nu[['offerdate', 'date']].values

array([['2013-04-24T00:00:00.000000000', '2012-03-02T00:00:00.000000000'],
       ['2013-04-24T00:00:00.000000000', '2012-03-02T00:00:00.000000000'],
       ['2013-04-24T00:00:00.000000000', '2012-03-06T00:00:00.000000000'],
       ...,
       ['2013-07-26T00:00:00.000000000', '2013-07-21T00:00:00.000000000'],
       ['2013-07-27T00:00:00.000000000', '2013-07-20T00:00:00.000000000'],
       ['2013-07-27T00:00:00.000000000', '2013-07-20T00:00:00.000000000']],
      dtype='datetime64[ns]')

In [ ]:
from datetime import datetime, timedelta

In [ ]:
trainhistory

In [ ]:
date_format = '%Y-%m-%d'
trainhistory['offerdate'].apply(lambda _: datetime.strptime(_, date_format))

0        2013-04-24
1        2013-03-27
2        2013-03-28
3        2013-03-25
4        2013-04-01
            ...    
160052   2013-04-22
160053   2013-04-24
160054   2013-04-02
160055   2013-04-17
160056   2013-04-05
Name: offerdate, Length: 160057, dtype: datetime64[ns]

In [ ]:
%%time
pd.to_datetime(nu['offerdate'], format = date_format, infer_datetime_format = True)

In [ ]:
pd.to_datetime([1, 2, 3], unit='D',
               origin=pd.Timestamp('1960-01-01'))

In [ ]:
nu['offerdate'].iloc[0]

'2013-04-24'

In [ ]:
pd.to_datetime(day, unit = 'D', origin = nu['offerdate'].iloc[0])

In [ ]:
day = np.linspace(30, 180, 6, endpoint = True)
date_format = '%Y-%m-%d'

In [ ]:
pd.to_datetime(nu['offerdate'], format = date_format) - pd.to_datetime(nu['date'], format = date_format)


id
86246        418 days
86246        418 days
86246        414 days
86246        413 days
86246        406 days
               ...   
4847628950     1 days
4847787712     5 days
4847787712     5 days
4853598737     7 days
4853598737     7 days
Length: 15349956, dtype: timedelta64[ns]

In [ ]:
mea = ['total', 'company', 'brand', 'category']

In [ ]:
sum(nu['offer' + mea[1]] == nu[mea[1]])

325558

In [ ]:
%%time
nu['total'] = 0
nu['offertotal'] = nu['total']

CPU times: user 44.1 ms, sys: 268 ms, total: 312 ms
Wall time: 321 ms


In [ ]:
%%time
day = np.linspace(30, 180, 6, endpoint = True).astype(int).astype(str)
date_format = '%Y-%m-%d'
nu['daydiff'] = pd.to_datetime(nu['offerdate'], format = date_format) - pd.to_datetime(nu['date'], format = date_format)
for i in day :
  nu['diff_' + i] = nu['daydiff'] <= timedelta(days = int(i))
for i in mea[1:] :
  nu['in_' + i] = nu['offer' + i] == nu[i]

CPU times: user 3.92 s, sys: 393 ms, total: 4.31 s
Wall time: 4.14 s


In [ ]:
%%time
a = nu[]

SyntaxError: ignored

In [ ]:
%%time
a = nu['productmeasure'] == nu['productmeasure']

CPU times: user 1.08 s, sys: 5 ms, total: 1.08 s
Wall time: 1.07 s


In [ ]:
nu.columns

Index(['chain', 'dept', 'category', 'company', 'brand', 'date', 'productsize',
       'productmeasure', 'purchasequantity', 'purchaseamount', 'offerdate',
       'offercategory', 'offercompany', 'offerbrand', 'daydiff', 'diff_30',
       'diff_60', 'diff_90', 'diff_120', 'diff_150', 'diff_180', 'in_company',
       'in_brand', 'in_category'],
      dtype='object')

In [ ]:
%%time
day = np.linspace(30, 180, 6, endpoint = True).astype(int).astype(str)
date_format = '%Y-%m-%d'
nu['offerdate'] = pd.to_datetime(nu['offerdate'], format = date_format, infer_datetime_format = True)
nu['date'] = pd.to_datetime(nu['date'], format = date_format, infer_datetime_format = True)
ddd = nu['offerdate']
nu['daydiff'] = ddd - nu['date']
for i in day :
  nu['diff_' + i] = nu['daydiff'] <= timedelta(days = int(i))

CPU times: user 3.99 s, sys: 1.47 s, total: 5.46 s
Wall time: 5.46 s


In [ ]:
nu.columns

Index(['chain', 'dept', 'category', 'company', 'brand', 'date', 'productsize',
       'productmeasure', 'purchasequantity', 'purchaseamount', 'offerdate',
       'offercategory', 'offercompany', 'offerbrand', 'daydiff', 'diff_30.0',
       'diff_60.0', 'diff_90.0', 'diff_120.0', 'diff_150.0', 'diff_180.0',
       'diff_30', 'diff_60', 'diff_90', 'diff_120', 'diff_150', 'diff_180'],
      dtype='object')

In [ ]:
range(1, 6)

range(1, 6)

In [ ]:
%time
of1 = offers[['offer', 'offervalue']]
use = pd.merge(use, of1, on='offer')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [ ]:
%%time
use1 = use.copy()

CPU times: user 4.73 ms, sys: 0 ns, total: 4.73 ms
Wall time: 4.76 ms


In [ ]:
# pip install pandarallel

In [ ]:
# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
%%time
test = group['chain'].count()
test.columns = ['id', 'buy_total_freq']

test = group['purchaseamount'].sum()
test.columns = ['id', 'buy_total_amount']

CPU times: user 184 ms, sys: 55.4 ms, total: 239 ms
Wall time: 244 ms


In [ ]:
import numpy as np

In [ ]:
%%time
test = group.agg({'purchaseamount': ['sum', 'mean'], 'daydiff': ['count']})
# test.columns = ['buy_total_amount', 'purchasequantity']

CPU times: user 317 ms, sys: 303 µs, total: 317 ms
Wall time: 316 ms


In [ ]:
test

In [ ]:
a = nu.groupby(['id', 'productmeasure'])['purchasequantity'].sum().reset_index('productmeasure')
# test.columns = ['buy_total_amount', 'purchasequantity']

In [ ]:
new2.iat[1, 1]

580

In [ ]:
new2.iloc[(meas == 'CT').tolist(), 1]

id
86246         6
86252         8
12262064      3
12682470      4
12996040      2
             ..
4806864099    7
4806866399    1
4813248649    4
4813618098    2
4847628950    1
Name: purchasequantity, Length: 122070, dtype: int64

In [ ]:
pd.concat([new1, new2.loc[meas == 'CT'],
           new2.loc[meas == 'OZ'],
           new2.loc[meas == 'LT']], axis = 1)

In [ ]:
meas = new2['productmeasure'].values

In [ ]:
%%time
# meas = new2['productmeasure'].values
aa = pd.concat([new1, new2.loc[(new2['productmeasure'].values == 'CT'), 'purchasequantity'],
                new2.loc[(new2['productmeasure'].values == 'OZ'), 'purchasequantity'],
                new2.loc[(new2['productmeasure'].values == 'LT'), 'purchasequantity']], axis = 1)

TypeError: ignored

In [ ]:
%%time
new1 = nu.groupby(['id']).agg({'purchaseamount': ['sum', 'mean', 'count'], 'daydiff': ['min']})
new1.columns = ['buy_total_amount', 'buy_total_amount', 'buy_total_freq', 'buy_total_daydiff']
new2 = nu.groupby(['id', 'productmeasure'])['purchasequantity'].sum().reset_index('productmeasure')
new = pd.concat([new1, new2.loc[(new2['productmeasure'].values == 'CT'), 'purchasequantity'],
                new2.loc[(new2['productmeasure'].values == 'OZ'), 'purchasequantity'],
                new2.loc[(new2['productmeasure'].values == 'LT'), 'purchasequantity']], axis = 1)

CPU times: user 2.28 s, sys: 452 ms, total: 2.73 s
Wall time: 2.71 s


In [ ]:
use1 = use.copy()

In [ ]:
mea = ['total', 'company', 'brand', 'category']

In [ ]:
%%time
use.set_index('id', inplace = True)

In [ ]:
%%time
use = use.set_index('id', inplace = True)
new1 = nu.groupby(['id']).agg({'purchaseamount': ['sum', 'mean', 'count'], 'daydiff': ['min']})
new2 = nu.groupby(['id', 'productmeasure'])['purchasequantity'].sum().reset_index('productmeasure')
new = pd.concat([new1, new2.loc[(new2['productmeasure'].values == 'CT'), 'purchasequantity'],
                new2.loc[(new2['productmeasure'].values == 'OZ'), 'purchasequantity'],
                new2.loc[(new2['productmeasure'].values == 'LT'), 'purchasequantity']], axis = 1)
new.columns = ['buy_total_amount', 'buy_total_avgamount', 'buy_total_freq',
               'buy_total_daydiff', 'buy_total_CT', 'buy_total_OZ', 'buy_total_LT']
use = use.join(new)

CPU times: user 2.42 s, sys: 446 ms, total: 2.87 s
Wall time: 2.86 s


In [ ]:
%%time
use1 = pd.concat([use1, new], axis = 1)

CPU times: user 100 ms, sys: 1.71 ms, total: 102 ms
Wall time: 97.6 ms


In [ ]:
%%time
use1 = pd.merge(use1, new, on = 'id')

CPU times: user 162 ms, sys: 0 ns, total: 162 ms
Wall time: 162 ms


In [ ]:
%%time
use1 = use1.set_index('id').join(new)

CPU times: user 62.9 ms, sys: 0 ns, total: 62.9 ms
Wall time: 62.6 ms


In [ ]:
nu.loc[nu['date'] >= nu[daa]].shape

(1157819, 27)

In [ ]:
nu.groupby(['id']).agg({'purchaseamount': ['sum', 'mean', 'count'], 'daydiff': ['min']})

In [ ]:
new1 = nu.loc[nu['date'] >= nu[daa]].groupby('id').agg({'purchaseamount' : ['sum', 'mean', 'count']})

In [ ]:
new1

In [ ]:
i = '30'
daa = 'diff_' + i

name = 'buy_total_amount_' + i
var = 'purchaseamount'

group = nu.loc[nu['date'] >= nu[daa]].groupby('id')
test = group[var].sum().reset_index()
test.columns = ['id', 'ascom', name]
use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')

In [ ]:
daa = 'diff_' + i
nu['ascom'] = nu['date'] >= nu[daa]

name = 'buy_total_amount_' + i
var = 'purchaseamount'

group = nu.groupby(['id', 'ascom'])
test = group[var].sum().reset_index()
test.columns = ['id', 'ascom', name]
use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')

In [ ]:
nu.groupby(['id', 'productmeasure'])['purchasequantity']

In [ ]:
nu['date'].values[0]

numpy.datetime64('2012-03-02T00:00:00.000000000')

In [ ]:
aa.iloc[:100]

id
86246   1 days
86246   1 days
86246   1 days
86246   1 days
86246   1 days
         ...  
86246   1 days
86246   1 days
86246   1 days
86246   1 days
86246   1 days
Name: daydiff, Length: 100, dtype: timedelta64[ns]

In [ ]:
nu.groupby('id')['date'].transform('max').head()

id
86246   2012-03-02
86246   2012-03-02
86246   2012-03-02
86246   2012-03-02
86246   2012-03-02
Name: date, dtype: datetime64[ns]

In [ ]:
nu['date']

id
86246        2012-03-02
86246        2012-03-02
86246        2012-03-06
86246        2012-03-07
86246        2012-03-14
                ...    
4847628950   2013-07-23
4847787712   2013-07-21
4847787712   2013-07-21
4853598737   2013-07-20
4853598737   2013-07-20
Name: date, Length: 15349956, dtype: datetime64[ns]

In [ ]:
newday = nu['date'].reset_index()

In [ ]:
for i in day :
  name = 'day_' + i
   = 

In [ ]:
newday

In [ ]:
%%time
new1 = nu[nu['diff_180']].groupby('id')
new2 = nu[nu['diff_180']].groupby(['id', 'productmeasure'])['purchasequantity']
# new1 = nu.loc[nu['date'] >= nu['diff_' + i]].groupby('id').agg({'purchaseamount' : ['sum', 'mean', 'count']})
# new2 = nu.loc[nu['date'] >= nu['diff_' + i]].groupby(['id', 'productmeasure'])['purchasequantity'].sum().reset_index('productmeasure')
# new = pd.concat([new1, new2.loc[(new2['productmeasure'].values == 'CT'), 'purchasequantity'],
#                 new2.loc[(new2['productmeasure'].values == 'OZ'), 'purchasequantity'],
#                 new2.loc[(new2['productmeasure'].values == 'LT'), 'purchasequantity']], axis = 1)
# new.columns = ['buy_total_amount_' + i, 'buy_total_avgamount_' + i, 'buy_total_freq_' + i,
#                 'buy_total_CT_' + i, 'buy_total_OZ_' + i, 'buy_total_LT_' + i]
# use = use.join(new)

CPU times: user 1.07 s, sys: 653 ms, total: 1.72 s
Wall time: 1.71 s


In [ ]:
aa = use[['chain', 'offer', 'market', 'offerdate', 'offervalue',
       'buy_total_amount', 'buy_total_avgamount', 'buy_total_freq',
       'buy_total_daydiff', 'buy_total_CT', 'buy_total_OZ', 'buy_total_LT']].copy()

In [ ]:
nu['diff_30']

In [ ]:
%%time
for i in day :
  filter = nu['diff_' + i]
  new1 = nu[filter].groupby('id').agg({'purchaseamount' : ['sum', 'mean', 'count']})
  new2 = nu[filter].groupby(['id', 'productmeasure'])['purchasequantity'].sum().reset_index('productmeasure')
  new = pd.concat([new1, new2[(new2['productmeasure'].values == 'CT')]['purchasequantity'],
                  new2[(new2['productmeasure'].values == 'OZ')]['purchasequantity'],
                  new2[(new2['productmeasure'].values == 'LT')]['purchasequantity']], axis = 1)
  new.columns = ['buy_total_amount_' + i, 'buy_total_avgamount_' + i, 'buy_total_freq_' + i,
                 'buy_total_CT_' + i, 'buy_total_OZ_' + i, 'buy_total_LT_' + i]
  use = use.join(new)

CPU times: user 9.79 s, sys: 2.24 s, total: 12 s
Wall time: 12 s


In [ ]:
nu.columns

Index(['chain', 'dept', 'category', 'company', 'brand', 'date', 'productsize',
       'productmeasure', 'purchasequantity', 'purchaseamount', 'offerdate',
       'offercategory', 'offercompany', 'offerbrand', 'daydiff', 'diff_30',
       'diff_60', 'diff_90', 'diff_120', 'diff_150', 'diff_180', 'in_company',
       'in_brand', 'in_category'],
      dtype='object')

In [ ]:
%%time
mea = ['company', 'brand', 'category']
for m in mea :
  filter = nu['in_' + m]
  new1 = nu[filter].groupby(['id']).agg({'purchaseamount': ['sum', 'mean', 'count'], 'daydiff': ['min']})
  new2 = nu[filter].groupby(['id', 'productmeasure'])['purchasequantity'].sum().reset_index('productmeasure')
  new = pd.concat([new1, new2.loc[(new2['productmeasure'].values == 'CT'), 'purchasequantity'],
                  new2.loc[(new2['productmeasure'].values == 'OZ'), 'purchasequantity'],
                  new2.loc[(new2['productmeasure'].values == 'LT'), 'purchasequantity']], axis = 1)
  new.columns = ['buy_'+m+'_amount', 'buy_'+m+'_avgamount', 'buy_'+m+'_freq',
                'buy_'+m+'_daydiff', 'buy_'+m+'_CT', 'buy_'+m+'_OZ', 'buy_'+m+'_LT']
  use = use.join(new)

CPU times: user 1.96 s, sys: 117 ms, total: 2.08 s
Wall time: 2.07 s


In [ ]:
%%time
for i in day :
  for m in mea :
    filter = nu['diff_'+i] & nu['in_' + m]
    new1 = nu[nu['diff_'+i]].groupby('id').agg({'purchaseamount' : ['sum', 'mean', 'count']})
    new2 = nu[nu['diff_'+i]].groupby(['id', 'productmeasure'])['purchasequantity'].sum().reset_index('productmeasure')
    new = pd.concat([new1, new2[(new2['productmeasure'].values == 'CT')]['purchasequantity'],
                    new2[(new2['productmeasure'].values == 'OZ')]['purchasequantity'],
                    new2[(new2['productmeasure'].values == 'LT')]['purchasequantity']], axis = 1)
    new.columns = ['buy_'+m+'_amount_' + i, 'buy_'+m+'_avgamount_' + i, 'buy_'+m+'_freq_' + i,
                  'buy_'+m+'_CT_' + i, 'buy_'+m+'_OZ_' + i, 'buy_'+m+'_LT_' + i]
    use = use.join(new)

In [ ]:
%%time
for i in day :
  filter = nu['diff_' + i]
  new1 = nu[filter].groupby('id').agg({'purchaseamount' : ['sum', 'mean', 'count']})
  new2 = nu[filter].groupby(['id', 'productmeasure'])['purchasequantity'].sum().reset_index('productmeasure')
  new = pd.concat([new1, new2[(new2['productmeasure'].values == 'CT')]['purchasequantity'],
                  new2[(new2['productmeasure'].values == 'OZ')]['purchasequantity'],
                  new2[(new2['productmeasure'].values == 'LT')]['purchasequantity']], axis = 1)
  new.columns = ['buy_total_amount_' + i, 'buy_total_avgamount_' + i, 'buy_total_freq_' + i,
                 'buy_total_CT_' + i, 'buy_total_OZ_' + i, 'buy_total_LT_' + i]

CPU times: user 10 s, sys: 1.37 s, total: 11.4 s
Wall time: 11.4 s


In [ ]:
it = np.repeat(range(3), 2)+1
it[:3]
it[3:]

array([2, 3, 3])

In [ ]:
%%time
use['not_buy_'+m] = True!=(use['buy_'+m+'_freq']>0)

CPU times: user 2.13 ms, sys: 0 ns, total: 2.13 ms
Wall time: 2.14 ms


In [ ]:
%%time
for m in mea :
  use['not_buy_'+m] = True!=(use['buy_'+m+'_freq']>0) 
  
it = np.repeat(range(3), 2)
for m1, m2 in zip(it[:3], it[3:]) :
  use['not_buy_'+mea[m1]+'_'+mea[m2]] = (use['not_buy_'+mea[m1]] & use['not_buy_'+mea[m2]])

CPU times: user 9.93 ms, sys: 1.02 ms, total: 11 ms
Wall time: 16.4 ms


In [ ]:
use[dayVar].fillna(timedelta(0)).astype('timedelta64[D]').astype(int)

In [ ]:
%%time
dayVar = ['buy_company_daydiff', 'buy_brand_daydiff', 'buy_category_daydiff', 'buy_total_daydiff']
use[dayVar] = use[dayVar].fillna(timedelta(0)).astype('timedelta64[D]').astype(int)
use = use.fillna(0)

CPU times: user 334 ms, sys: 83.3 ms, total: 418 ms
Wall time: 418 ms


In [ ]:
%%time
# for i in use[dayVar] :
#   use[i] = use[i].astype('str').apply(lambda x:x[:-5]).astype('int32')
# transform bool type into int
for i in use.columns :
  if(use[i].dtypes == 'int') :
    use[i] = use[i].apply(bool)

CPU times: user 540 ms, sys: 0 ns, total: 540 ms
Wall time: 544 ms


In [ ]:
%%time
use[use.columns[use.dtypes == 'bool']] = use[use.columns[use.dtypes == 'bool']].astype(int)

In [ ]:
for i in use.columns :
  if(use[i].dtypes == 'bool') :
    use[i] = use[i].apply(int)

In [ ]:
# generate not buy index
nu['ascom'] = (nu['company'] == nu['offercompany']) & (nu['brand'] == nu['offerbrand']) & (nu['category'] == nu['offercategory'])
group = nu.groupby(['id', 'ascom'])

name1 = 'buy_company_brand_category'
name = name1 + '_freq'

test = group['chain'].count().reset_index()
test.columns = ['id', 'ascom', name]
use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
new = pd.DataFrame((use[name] > 0) != True)
new.columns = ['not_' + name1]
use = pd.concat([use, new], axis = 1)



nu['ascom'] = (nu['company'] == nu['offercompany']) & (nu['brand'] == nu['offerbrand'])
group = nu.groupby(['id', 'ascom'])

name1 = 'buy_company_brand'
name = name1 + '_freq'

test = group['chain'].count().reset_index()
test.columns = ['id', 'ascom', name]
use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
new = pd.DataFrame((use[name] > 0) != True)
new.columns = ['not_' + name1]
use = pd.concat([use, new], axis = 1)



nu['ascom'] = (nu['company'] == nu['offercompany']) & (nu['category'] == nu['offercategory'])
group = nu.groupby(['id', 'ascom'])

name1 = 'buy_company_category'
name = name1 + '_freq'

test = group['chain'].count().reset_index()
test.columns = ['id', 'ascom', name]
use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
new = pd.DataFrame((use[name] > 0) != True)
new.columns = ['not_' + name1]
use = pd.concat([use, new], axis = 1)



nu['ascom'] = (nu['brand'] == nu['offerbrand']) & (nu['category'] == nu['offercategory'])
group = nu.groupby(['id', 'ascom'])

name1 = 'buy_brand_category'
name = name1 + '_freq'

test = group['chain'].count().reset_index()
test.columns = ['id', 'ascom', name]
use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
new = pd.DataFrame((use[name] > 0) != True)
new.columns = ['not_' + name1]
use = pd.concat([use, new], axis = 1)

del new
gc.collect()

new1 = pd.DataFrame((use['buy_company_freq'] > 0) != True)
new1.columns = ['not_buy_company']
new2 = pd.DataFrame((use['buy_brand_freq'] > 0) != True)
new2.columns = ['not_buy_brand']
new3 = pd.DataFrame((use['buy_category_freq'] > 0) != True)
new3.columns = ['not_buy_category']
use = pd.concat([use, new1, new2, new3], axis = 1)

del new1, new2, new3
gc.collect()

# handle na problem
dayVar = ['buy_company_daydiff', 'buy_brand_daydiff', 'buy_category_daydiff', 'buy_total_daydiff']
use1 = use[dayVar]
use.drop(columns = dayVar, inplace = True)
use = use.fillna(0)
use1 = use1.fillna(timedelta(0))
use = pd.concat([use, use1], axis = 1)

del use1
gc.collect()

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(handle na problem) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1

# transform date type into int
for i in use[dayVar] :
  use[i] = use[i].astype('str').apply(lambda x:x[:-5]).astype('int32')

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(transform date type) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1

# transform bool type into int
for i in use.columns :
  if(use[i].dtypes == 'bool') :
    use[i] = use[i].apply(int)

In [ ]:
# import useful package
from datetime import timedelta
from itertools import cycle
import pandas as pd
import numpy as np
import time
import gc
ts = time.time()
# tk = 1

# get all data & delete those not in transactions
train = trainhistory.drop(columns = ['repeater', 'repeattrips'])
data = pd.concat([train, testhistory], axis=0, ignore_index=True)
use = data[data['id'].isin(transactions['id'])] # (310665, )

del trainhistory, train, testhistory
gc.collect()

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(get all data) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1

# put offer information into transactions
of = offers[['offer', 'category', 'company', 'offervalue', 'brand']]
usf = pd.merge(use, of, on='offer')
usf.columns = ['id', 'chain', 'offer', 'market', 'offerdate', 'offercategory', 'offercompany',
      'offervalue', 'offerbrand']
tu = usf[['id', 'offer', 'offerdate', 'offercategory', 'offercompany', 'offerbrand']]
nu = pd.merge(tu, transactions, on='id')

del tu, usf, of, transactions
gc.collect()

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(put offer information) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1

# generate time index
date_format = '%Y-%m-%d'
nu['offerdate'] = pd.to_datetime(nu['offerdate'], format = date_format)
nu['date'] = pd.to_datetime(nu['date'], format = date_format)
nu['daydiff'] = nu['offerdate'] - nu['date']
nu['diff_180'] = nu['offerdate'] - timedelta(days = 180)
nu['diff_150'] = nu['offerdate'] - timedelta(days = 150)
nu['diff_120'] = nu['offerdate'] - timedelta(days = 120)
nu['diff_90'] = nu['offerdate'] - timedelta(days = 90)
nu['diff_60'] = nu['offerdate'] - timedelta(days = 60)
nu['diff_30'] = nu['offerdate'] - timedelta(days = 30)

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate time index) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1

# put offervalue
of1 = offers[['offer', 'offervalue']]
use = pd.merge(use, of1, on='offer')

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(put offervalue) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1

# generate total
group = nu.groupby(['id'])

test = group['chain'].count().reset_index()
test.columns = ['id', 'buy_total_freq']
use = pd.merge(use, test, on='id')

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total freq) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1

test = group['purchaseamount'].sum().reset_index()
test.columns = ['id', 'buy_total_amount']
use = pd.merge(use, test, on='id')

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total amount) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1

test = group['purchaseamount'].mean().reset_index()
test.columns = ['id', 'buy_total_avgamount']
use = pd.merge(use, test, on='id')

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total avg) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1

test = group['purchasequantity'].sum().reset_index()
test.columns = ['id', 'buy_total_quantity']
use = pd.merge(use, test, on='id')

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total quan) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1

test = group['daydiff'].min().reset_index()
test.columns = ['id', 'buy_total_daydiff']
use = pd.merge(use, test, on='id')

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate total) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1


day = np.linspace(30, 180, 6, endpoint=True).astype(int).astype(str)

for i in day :
  daa = 'diff_' + i
  nu['ascom'] = nu['date'] >= nu[daa]

  name = 'buy_total_amount_' + i
  var = 'purchaseamount'

  group = nu.groupby(['id', 'ascom'])
  test = group[var].sum().reset_index()
  test.columns = ['id', 'ascom', name]
  use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')

  # te = time.time()
  # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate day amount) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
  # tk+=1

  name = 'buy_total_quantity_' + i
  var = 'purchasequantity'

  group = nu.groupby(['id', 'ascom'])
  test = group[var].sum().reset_index()
  test.columns = ['id', 'ascom', name]
  use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')

  # te = time.time()
  # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate day quan) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
  # tk+=1

  name = 'buy_total_freq_' + i
  var = 'purchaseamount'

  group = nu.groupby(['id', 'ascom'])
  test = group['chain'].count().reset_index()
  test.columns = ['id', 'ascom', name]
  use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')

  # te = time.time()
  # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate day) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
  # tk+=1

# generate company & brand & category
mea = ['company', 'brand', 'category']
day = np.repeat(np.linspace(30, 180, 6, endpoint=True), 3).astype(int).astype(str)

for i in mea :
  nu['ascom'] = nu[i] == nu['offer' + i]
  group = nu.groupby(['id', 'ascom'])

  test = group['chain'].count().reset_index()
  test.columns = ['id', 'ascom', 'buy_'+i+'_freq']
  use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_freq']], on='id', how = 'outer')

  # te = time.time()
  # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other freq) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
  # tk+=1

  test = group['purchaseamount'].sum().reset_index()
  test.columns = ['id', 'ascom', 'buy_'+i+'_amount']
  use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_amount']], on='id', how = 'outer')

  # te = time.time()
  # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other amount) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
  # tk+=1

  test = group['purchaseamount'].mean().reset_index()
  test.columns = ['id', 'ascom', 'buy_'+i+'_avgamount']
  use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_avgamount']], on='id', how = 'outer')

  # te = time.time()
  # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other avg) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
  # tk+=1

  test = group['purchasequantity'].sum().reset_index()
  test.columns = ['id', 'ascom', 'buy_'+i+'_quantity']
  use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_quantity']], on='id', how = 'outer')

  # te = time.time()
  # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other quan) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
  # tk+=1

  test = group['daydiff'].min().reset_index()
  test.columns = ['id', 'ascom', 'buy_'+i+'_daydiff']
  use = pd.merge(use, test[test['ascom']][['id', 'buy_'+i+'_daydiff']], on='id', how = 'outer')

  # te = time.time()
  # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
  # tk+=1


for i, j in zip(day, cycle(mea)) :
  daa = 'diff_' + i
  nu['ascom'] = ((nu['date'] >= nu[daa]) & (nu[j] == nu['offer' + j]))

  name = 'buy_'+j+'_amount_' + i
  var = 'purchaseamount'

  group = nu.groupby(['id', 'ascom'])
  test = group[var].sum().reset_index()
  test.columns = ['id', 'ascom', name]
  use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')

  # te = time.time()
  # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other day amount) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
  # tk+=1

  name = 'buy_'+j+'_quantity_' + i
  var = 'purchasequantity'

  group = nu.groupby(['id', 'ascom'])
  test = group[var].sum().reset_index()
  test.columns = ['id', 'ascom', name]
  use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')

  # te = time.time()
  # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other day quan) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
  # tk+=1

  name = 'buy_'+j+'_freq_' + i
  var = 'purchaseamount'

  group = nu.groupby(['id', 'ascom'])
  test = group['chain'].count().reset_index()
  test.columns = ['id', 'ascom', name]
  use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')

  # te = time.time()
  # # print('Inner -', str(tidx) + ' of ' + str(tk) + '(generate other day) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
  # # tk+=1

# generate not buy index
nu['ascom'] = (nu['company'] == nu['offercompany']) & (nu['brand'] == nu['offerbrand']) & (nu['category'] == nu['offercategory'])
group = nu.groupby(['id', 'ascom'])

name1 = 'buy_company_brand_category'
name = name1 + '_freq'

test = group['chain'].count().reset_index()
test.columns = ['id', 'ascom', name]
use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
new = pd.DataFrame((use[name] > 0) != True)
new.columns = ['not_' + name1]
use = pd.concat([use, new], axis = 1)



nu['ascom'] = (nu['company'] == nu['offercompany']) & (nu['brand'] == nu['offerbrand'])
group = nu.groupby(['id', 'ascom'])

name1 = 'buy_company_brand'
name = name1 + '_freq'

test = group['chain'].count().reset_index()
test.columns = ['id', 'ascom', name]
use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
new = pd.DataFrame((use[name] > 0) != True)
new.columns = ['not_' + name1]
use = pd.concat([use, new], axis = 1)



nu['ascom'] = (nu['company'] == nu['offercompany']) & (nu['category'] == nu['offercategory'])
group = nu.groupby(['id', 'ascom'])

name1 = 'buy_company_category'
name = name1 + '_freq'

test = group['chain'].count().reset_index()
test.columns = ['id', 'ascom', name]
use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
new = pd.DataFrame((use[name] > 0) != True)
new.columns = ['not_' + name1]
use = pd.concat([use, new], axis = 1)



nu['ascom'] = (nu['brand'] == nu['offerbrand']) & (nu['category'] == nu['offercategory'])
group = nu.groupby(['id', 'ascom'])

name1 = 'buy_brand_category'
name = name1 + '_freq'

test = group['chain'].count().reset_index()
test.columns = ['id', 'ascom', name]
use = pd.merge(use, test[test['ascom']][['id', name]], on='id', how = 'outer')
new = pd.DataFrame((use[name] > 0) != True)
new.columns = ['not_' + name1]
use = pd.concat([use, new], axis = 1)

del new
gc.collect()

new1 = pd.DataFrame((use['buy_company_freq'] > 0) != True)
new1.columns = ['not_buy_company']
new2 = pd.DataFrame((use['buy_brand_freq'] > 0) != True)
new2.columns = ['not_buy_brand']
new3 = pd.DataFrame((use['buy_category_freq'] > 0) != True)
new3.columns = ['not_buy_category']
use = pd.concat([use, new1, new2, new3], axis = 1)

del new1, new2, new3
gc.collect()

# handle na problem
dayVar = ['buy_company_daydiff', 'buy_brand_daydiff', 'buy_category_daydiff', 'buy_total_daydiff']
use1 = use[dayVar]
use.drop(columns = dayVar, inplace = True)
use = use.fillna(0)
use1 = use1.fillna(timedelta(0))
use = pd.concat([use, use1], axis = 1)

del use1
gc.collect()

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(handle na problem) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1

# transform date type into int
for i in use[dayVar] :
  use[i] = use[i].astype('str').apply(lambda x:x[:-5]).astype('int32')

# te = time.time()
# print('Inner -', str(tidx) + ' of ' + str(tk) + '(transform date type) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')
# tk+=1

# transform bool type into int
for i in use.columns :
  if(use[i].dtypes == 'bool') :
    use[i] = use[i].apply(int)

te = time.time()
print('Inner -', str(tidx) + ' (generate feature) -> time elapsed: ' + str(round(te-ts, 2)) + ' seconds')

return use

In [ ]:
target_dates = pd.date_range(end=now, periods=100, freq="7D")
full_dates = pd.date_range(start, now).tolist()
org_dates = df.date.tolist()

last_date = None
for d in full_dates:
 if d in org_dates:
  date_map[d] = d
  last_date = d
 elif last_date is not None:
  date_map[d] = last_date
 else:
  continue
  
#### main change is from here #####
date_map_list = []
for td in target_dates:
 date_map_list.append({"target_date":td, "org_date":date_map[td]}) 
date_map_df = pd.DataFrame(date_map_list)
new_df = date_map_df.merge(df, left_on=["org_date"], right_on=["date"], how="inner")